In [1]:
from langchain.tools import Tool
from langchain_core.tools import tool

In [2]:
from langchain.agents import AgentExecutor, create_react_agent,initialize_agent,AgentType
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from langchain.prompts import PromptTemplate, ChatPromptTemplate

In [3]:
from add import add_two_numbers

In [4]:
# Set API Key
os.environ["GOOGLE_API_Key"] = os.getenv("Gemini_API_KEY")

In [5]:
# Define the system prompt template with required variables
system_template = """You are a helpful AI assistant. Use the following tools to answer the user's question:

{tools}

When asked a question, always follow this format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

{agent_scratchpad}
"""


In [6]:
# Create a chat prompt template 
prompt = ChatPromptTemplate.from_messages([
    ("system", system_template),
    ("human", "{input}")
])

In [7]:

# Initialize Gemini
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro-latest",
    temperature=0.7,
    convert_system_message_to_human=True
)

In [8]:
@tool
def square_number(number: str) -> int:
    """Returns the square of a given number."""
    return int(number)** 2

In [15]:
@tool
def add_two_numbers(numbers: str) -> int:
    """Parses two comma-separated numbers and returns their sum."""
    try:
        num1, num2 = map(int, numbers.split(","))
        return num1 + num2
    except Exception as e:
        return f"Error: {e}. Please provide two numbers in the format '5,6'."

In [16]:
# Use create_react_agent with the chat prompt
agent = create_react_agent(
    llm=llm,
    tools=[square_number,add_two_numbers],
    prompt=prompt
)


In [11]:
# Create agent executor
agent_executor = AgentExecutor.from_agent_and_tools(
    agent=agent,
    tools=[square_number,add_two_numbers],
    verbose=True,
    handle_parsing_errors=True
)

In [17]:
response = agent_executor.invoke({"input": "what is the sum of 5 and 6"})



> Entering new AgentExecutor chain...


c:\Users\karti\Desktop\Project\Data_analyst_agent\myenv\Lib\site-packages\langchain_google_genai\chat_models.py:362: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Question: what is the sum of 5 and 6
Thought: I should use the add_two_numbers tool to get the sum of 5 and 6.
Action: add_two_numbers
Action Input: 5, 6

ValidationError: 2 validation errors for add_two_numbers
number1
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='5, 6', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/int_parsing
number2
  Field required [type=missing, input_value={'number1': '5, 6'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [ ]:
print(response)

In [ ]:
from IPython.display import display

display(response["output"]) 